In [0]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# Load data
pdf = pd.read_csv("/databricks-datasets/wine-quality/winequality-red.csv",sep=";")
X = pdf.drop("quality", axis=1)
y = pdf["quality"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Predict and get metrics
preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds, squared=False)

# Define conda env explicitly
conda_env = {
    "name": "mlflow-env",
    "channels": ["defaults"],
    "dependencies": [
        "python=3.11",
        "scikit-learn=1.3.0",
        "pip",
        {
            "pip": ["mlflow"]
        }
    ]
}

import mlflow.models
from mlflow.models.signature import infer_signature

signature = infer_signature(X_test, preds)

mlflow.set_registry_uri("databricks-uc")

with mlflow.start_run() as run:
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        conda_env=conda_env,
        signature=signature,
        registered_model_name="ml.naval.wine_quality_model_uc"
    )
